## 파이썬과 구글 스프레드시트를 이용하여 운동 추적 앱 만들기 
- chat GPT를 이용해서 어떤 운동을 했는지 기록하는 어플 만들기  
    - 예: ran 5k for 28 minutes.
- 위의 영어문장을 쓰면, 운동 종류, 운동 시간, 등의 정보를 구글 스프레드시트에 저장할 수 있게 하기
- 힌트: datetime strftime(), requests API를 사용하기 

### 1. 1단계 - API 인증과 구글 스프레드시트 준비하기 
1. ‘나의 운동(My Workouts)’ 스프레드시트의 사본 만들기(본인 구글 계정으로 로그인해야 함.)
2. [뉴트리셔닉스(Nutritionix](https://www.nutritionix.com/) API 웹사이트로 가서 ‘API key 얻기’를 선택하여 무료 가입 하기. 
    - 만약 뉴트리셔닉스의 인증 메일이 오지 않는다면 스팸함을 확인해 보기(Gmail이라면 ‘프로모션’탭도 확인).
3. 로그인 후 API Key와 App Id 접근이 가능, 해당 API Key와 App ID 확인 후 저장히기 
    - [뉴트리셔닉스 API](https://developer.nutritionix.com/)

### 2. 2단계 - 자연어 쿼리로 운동 통계 얻기 
- 뉴트리셔닉스 ‘운동용 자연어’ API 문서를 보고, 텍스트로 간단하게 운동을 입력했을 때, 운동 통계를 출력하는 방법을 파악.
    - API Document - https://docs.google.com/document/d/1_q-K-ObMTZvO0qUEAxROrN3bwMujwAN25sLHwJzliK0/edit#heading=h.zhjgcprrgvim

In [5]:
import requests      

# 파라미터에 들어갈 상수 값들 
GENDER = Your Gender        # 성별 
WEIGHT_KG = Your Weight     # 몸무게 
HEIGHT_CM = Your Height     # 키 
AGE = Your Age              # 나이 

APP_ID = "Your App ID"        # 뉴트리셔닉스 App ID 
API_KEY = "Your API Key"      # 뉴트리셔닉스 API Key 

END_POINT = "https://trackapi.nutritionix.com/v2/natural/exercise"  # 엔드 포인트 
  
exercise_text = input("Tell me which exercise you did: ")     # 오늘 운동 뭐했는지 입력 후 저장 - 파라미터 쿼리 값 

# 인증을 위해 해더 값 입력 
headers = {
    "x-app-id": APP_ID,      # 뉴트리셔닉스 App ID
    "x-app-key": API_KEY,    # 뉴트리셔닉스 API Key 
}

# 파라미터 값 입력 
parameters = {
    "query": exercise_text,       # 쿼리 
    "gender": GENDER,             # 성별  
    "weight_kg": WEIGHT_KG,       # 몸무게 
    "height_cm": HEIGHT_CM,       # 키 
    "age": AGE                    # 나이
}

# 데이터 불러오기 
response = requests.post(url=END_POINT, json=parameters, headers=headers)   # Post 방식으로 데이터 불러오기 
result = response.json()              # Post 방식으로 불러온 데이터 JSON 형태로 반환                       
print(result)                         # 결과 출력

Tell me which exercise you did: ran 5km
{'exercises': [{'tag_id': 317, 'user_input': 'ran', 'duration_min': 31.08, 'met': 9.8, 'nf_calories': 383.78, 'photo': {'highres': 'https://d2xdmhkmkbyw75.cloudfront.net/exercise/317_highres.jpg', 'thumb': 'https://d2xdmhkmkbyw75.cloudfront.net/exercise/317_thumb.jpg', 'is_user_uploaded': False}, 'compendium_code': 12050, 'name': 'running', 'description': None, 'benefits': None}]}


### 3. 3단계 - Sheety로 구글시트 준비하기 
1. [sheety](https://sheety.co/)에 로그인 하기 
    - 중요: 1단계에서 구글 시트를 만들었던 계정과 동일한 계정이여야 함) 
2. 프로젝트 페이지에서 ‘새 프로젝트’를 클릭하여, Sheety내에 새 프로젝트 ‘운동 추적(My Workout)’을 생성한 후, ‘나의 운동’ 구글 시트의 URL을 붙여 넣기.
3. 프로젝트 생성후 API End point를 클릭하고 GET, POST 두 개에 체크하기 

### 4. 4단계 - 구글시트에 데이터 저장하기 
1. [sheety API 문서](https://sheety.co/docs/requests)를 활용하여, Sheety API를 사용한 코드를 작성하여 뉴트리셔닉스 API로 얻은 운동 데이터를 구글 시트의 신규 행으로 추가하기. 

In [22]:
from datetime import datetime

# sheety endpoint 
SHEETY_ENDPOINT = "https://api.sheety.co/52479a109dad9a29f7d2ae8bbb70d1a7/myWorkouts/workouts"

# 시간 설정 
now = datetime.now()                   # datetime 객체 선언 - 현재 시간으로 
today_date = now.strftime("%Y/%m/%d")  # 날짜 설정 
today_time = now.strftime("%X")        # 시간 설정 

# 추출한 데이터 구글시트에 저장 
for exercise in result["exercises"]:  
    sheet_input = {
        "workout": {                   # 위에 입력한 workout 데이터 
            "date": today_date,        # 금일 날짜 
            "time": today_time,        # 업로드 시간
            "exercise": exercise["name"].title(),   # 운동명 
            "duration": exercise["duration_min"],   # 지속 시간 
            "calories": exercise["nf_calories"]     # 운동 칼로리
        }
    }
    
    # 데이터 입력을 위해 POST 방식으로 입력 
    sheet_response = requests.post(url=SHEETY_ENDPOINT, json=sheet_input)
    
    # 결과 추출
    print(sheet_response.text)

{
  "workout": {
    "date": "2023/08/20",
    "time": "21:46:37",
    "exercise": "Running",
    "duration": 31.08,
    "calories": 383.78,
    "id": 2
  }
}


### 5. 5단계 - Sheety API 인증하기 
1. Sheety 엔드포인트를 보호하기 위해 ‘Basic 인증’이나 ‘Bearer 토큰’을 추가하기. 
    - Bearer 인증이란? : Bearer 인증(혹은 토큰 인증)은 보안 토큰을 포함하는 HTTP 인증 방식이다. ‘Bearer 인증’이라는 이름에서부터 ‘이 토큰을 나르는 사람에게 접근 권한을 부여한다’는 의미이다. 보안 토큰, 혹은 ‘bearer 토큰’은 암호화된 문자열이며, 다음과 같은 것들이 Bearer 토큰의 예시이다. 
        - "AAAAAAAAAAAAAAAAAAAAAMLheAAAAAAA0%2BuSeid%2BULvsea4JtiGRiSDSJSI%3DEUifiRBkKG5E2XzMDjRfl76ZC9Ub0wnz4XsNiRVBChTYbJcE3F"
    - 보안 토큰을 가진 사람은 누구나 스프레드시트에 접근할 수 있도록 허락된다는 개념이다. 브라우저나 모바일 앱과 같은 클라이언트가 Sheety 서버로 요청을 보낼 때, 인증 헤더에 이 보안 토큰을 넣어 보낸다.
2. 인증에 관한 Sheety 문서를 보고, 파이썬 코드를 수정하여 요청을 인증하기 

In [ ]:
sheet_response = requests.post(sheet_endpoint, json=sheet_inputs)
  
#Basic Authentication
sheet_response = requests.post(
  sheet_endpoint, 
  json=sheet_inputs, 
  auth=(
      YOUR USERNAME, 
      YOUR PASSWORD,
  )
)

#Bearer Token Authentication
bearer_headers = {
"Authorization": "Bearer YOUR_TOKEN"
}
sheet_response = requests.post(
    sheet_endpoint, 
    json=sheet_inputs, 
    headers=bearer_headers
)

### 총 정리 코드 

In [ ]:
import requests      
from datetime import datetime

# 파라미터에 들어갈 상수 값들 
GENDER = Your Gender        # 성별 
WEIGHT_KG = Your Weight     # 몸무게 
HEIGHT_CM = Your Height     # 키 
AGE = Your Age              # 나이 

# USER 및 API 정보 
APP_ID = "Your App ID"        # 뉴트리셔닉스 App ID 
API_KEY = "Your API Key"      # 뉴트리셔닉스 API Key 

END_POINT = "https://trackapi.nutritionix.com/v2/natural/exercise"  # nutritionix endpoint
SHEETY_ENDPOINT = "https://api.sheety.co/52479a109dad9a29f7d2ae8bbb70d1a7/myWorkouts/workouts"  # sheety endpoint 

# 시간 설정 
now = datetime.now()                   # datetime 객체 선언 - 현재 시간으로 
today_date = now.strftime("%Y/%m/%d")  # 날짜 설정 
today_time = now.strftime("%X")        # 시간 설정 

exercise_text = input("Tell me which exercise you did: ")     # 오늘 운동 뭐했는지 입력 후 저장 - 파라미터 쿼리 값 

# 인증을 위해 해더 값 입력 
headers = {
    "x-app-id": APP_ID,      # 뉴트리셔닉스 App ID
    "x-app-key": API_KEY,    # 뉴트리셔닉스 API Key 
}

# 파라미터 값 입력 
parameters = {
    "query": exercise_text,       # 쿼리 
    "gender": GENDER,             # 성별  
    "weight_kg": WEIGHT_KG,       # 몸무게 
    "height_cm": HEIGHT_CM,       # 키 
    "age": AGE                    # 나이
}

# 데이터 불러오기 
response = requests.post(url=END_POINT, json=parameters, headers=headers)   # Post 방식으로 데이터 불러오기 
result = response.json()              # Post 방식으로 불러온 데이터 JSON 형태로 반환                       

# 추출한 데이터 구글시트에 저장 
for exercise in result["exercises"]:  
    sheet_input = {
        "workout": {                   # 위에 입력한 workout 데이터 
            "date": today_date,        # 금일 날짜 
            "time": today_time,        # 업로드 시간
            "exercise": exercise["name"].title(),   # 운동명 
            "duration": exercise["duration_min"],   # 지속 시간 
            "calories": exercise["nf_calories"]     # 운동 칼로리
        }
    }
    
    # 데이터 입력을 위해 POST 방식으로 입력 
    sheet_response = requests.post(url=SHEETY_ENDPOINT, json=sheet_input)
    
    # 결과 추출
    print(sheet_response.text)